# Microbiome Analysis in R

This tutorial provides a comprehensive overview of the main functions of the [microeco](https://github.com/ChiLiubio/microeco) library. This R package is designed to assist users in analyzing microbiome data using a wide range of methodologies that enhance the analytical experience.

Throughout the tutorial, you will learn how to load data into R, manipulate and normalize it, calculate relative abundances, perform descriptive analyses based on those abundances, carry out diversity analyses (alpha and beta indices), and represent the results graphically.

### Setup

In [ ]:
# Download the r2u File
download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")
Sys.chmod("add_cranapt_jammy.sh", "0755")
system("./add_cranapt_jammy.sh")
bspm::enable()
options(bspm.version.check=FALSE)
system("rm add_cranapt_jammy.sh")

In [ ]:
# Shell_call function
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

> **Note**: This setup is only necessary for Google Colab. DO NOT use it in R or RStudio.

## 1. Package Installation

Before proceeding with the installation, it is important to note that, like many other R libraries, **microeco** is a package composed of various dependencies. These dependencies are other R libraries that must be installed beforehand to use **microeco** properly.

The command `install.packages("BiocManager")` in R is used to install the [BiocManager](https://www.bioconductor.org/install/) package from CRAN. This package is an essential tool for working with packages from the [Bioconductor](https://www.bioconductor.org/) project, which provides bioinformatics tools for the analysis and interpretation of biological data. Bioconductor is a specialized repository offering bioinformatics and statistical tools designed for the analysis of genomic, transcriptomic, proteomic data, and more.

In [ ]:
# Installing BiocManager

install.packages("BiocManager")

In [ ]:
# Installing Packages with BiocManager

BiocManager::install("ggtree")
BiocManager::install("metagenomeSeq")
BiocManager::install("ALDEx2")
BiocManager::install("ANCOMBC")
install.packages("file2meco", repos = BiocManager::repositories())
install.packages("MicrobiomeStat", repos = BiocManager::repositories())
install.packages("WGCNA", repos = BiocManager::repositories())
install.packages("remotes")
install.packages("devtools")
install.packages("picante")
install.packages("ggpubr")

In [ ]:
# Installing Packages

install.packages("file2meco", repos = BiocManager::repositories())
install.packages("MicrobiomeStat", repos = BiocManager::repositories())
install.packages("WGCNA", repos = BiocManager::repositories())
install.packages("remotes")
install.packages("devtools")
remotes::install_github("jbisanz/qiime2R")

> **Note:** It is common for R libraries to undergo constant updates, so there may be changes that are not reflected in this guide. It is recommended to visit the following page in case of any issues: [https://chiliubio.github.io/microeco_tutorial/](https://chiliubio.github.io/microeco_tutorial/) as well as [https://github.com/ChiLiubio/microeco](https://github.com/ChiLiubio/microeco) for the library installation.

Finally, proceed with the installation of the *microeco* package.

In [ ]:
# Installing microeco Packages Directly from a GitHub Repository

devtools::install_github("ChiLiubio/microeco")

Once the packages or libraries are downloaded, we will load them using the `library()` function.

In [ ]:
# Load Libraries

library(microeco)
library(qiime2R)
library(ggplot2)
library(magrittr)
library(RColorBrewer)
library(ggalluvial)
library(picante)
library(ggpubr)

## 2. Download Data

The data used in this tutorial is available in a repository on Zenodo: [https://zenodo.org/records/13972776](https://zenodo.org/records/13972776).

To download the data, we will use the R function `download.file()`:

In [ ]:
urls <- c(
  "https://zenodo.org/records/13972776/files/classification.qza",
  "https://zenodo.org/records/13972776/files/sample-metadata.tsv",
  "https://zenodo.org/records/13972776/files/table.qza",
  "https://zenodo.org/records/13972776/files/rooted_tree.qza"
)

# Name files
destfiles <- c("classification.qza", "sample-metadata.tsv", "table.qza", "rooted_tree.qza")

# Download files
for (i in seq_along(urls)) {
  download.file(url = urls[i], destfile = destfiles[i], mode = "wb")
  cat(sprintf("File %s downloaded.\n", destfiles[i]))
}

You will obtain the following result:

![descarga](./Images/download.png)

## 3. Load Data

To load the data, we will use an R library designed for working with files produced by the [QIIME2](https://qiime2.org/) program. This library is called **qiime2R**, which has already been downloaded and loaded in the previous steps.

Next, use the `setwd()` function to set the working directory to the location where the data files are stored. These files are available in the tutorial folder.

In [ ]:
# Select the Directory

setwd("/content")

In [ ]:
# Load Data

datos= qza_to_phyloseq(features = "table.qza",
  taxonomy =   "classification.qza",
  tree = "rooted_tree.qza",
  metadata = "sample-metadata.tsv")

We extract each of the data files to implement them in the functions of *microeco*.

In [ ]:
otu_table = as.data.frame(datos@otu_table)
metadatos = data.frame(datos@sam_data)
arbol = datos@phy_tree
taxonomia = data.frame(datos@tax_table)

Using the `microtable$new` function, a new dataset can be generated with the loaded files. The main difference is that this object is compatible with the other functions of the library.

In [ ]:
# Data File for microtable (microeco)
dataset <-
  microtable$new(
    otu_table = otu_table,
    sample_table = metadatos,
    phylo_tree = arbol,
    tax_table = taxonomia

## 4. Rarefaction

A common term in microbiome data analysis is *rarefaction*. This refers to a random sampling process applied to the units contained in the OTUs (Operational Taxonomic Units), where a value must be defined for sampling and standardizing the units. Usually, the selected value is that of the sample with the smallest number of sequences. To verify the range of sequences in the OTUs, the first line of code is used, which indicates that the sample with the fewest sequences contains 1067 sequences. For this reason, rarefaction is performed by fixing this value and using the `rarefy_samples` function. As it is a random sampling process, a seed must be set to replicate the results.

In [ ]:
# Set a Seed to Replicate the Results

set.seed(1)

In [ ]:
# Verification of OTU Value Range

dataset$sample_sums() %>% range

In [ ]:
# Rarefaction

dataset$rarefy_samples(sample.size = 1067)

> **Note:** The application of the rarefaction technique has been questioned over the years in microbiome studies. To gain a deeper understanding of this discussion, it is recommended to read the following article: [https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003531](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003531)

## 4. Calculation of Relative Abundance

One of the most important aspects of microbiome analysis is identifying the main bacteria present in the study units. For this, relative abundance is used, which indicates the presence of each bacterium in each study unit on a scale from 0 to 1.

The following function allows for the calculation of abundance at different levels of the phylogeny. This abundance can be accessed via `dataset$taxa_abund`. By adding another `$` after `taxa_abund`, you can access the abundance at various taxonomic levels, as shown in the second line of code.

In [ ]:
# Calculation of Abundance

dataset$cal_abund()

In [ ]:
# Accessing the Data
# [1:2,1:2] is used to display the first two rows and columns (representing the first two samples and bacteria).

dataset$taxa_abund$Family[1:2,1:2]

## 5. Data Manipulation Functions

### 5.1 Taxonomy Grouping

In previous tutorials, OTUs could be grouped at various levels of the phylogeny, but this required using multiple libraries. The *microtable* library allows for grouping the OTUs using the `merge_taxa` function. This will adjust the information in the OTU table based on the desired taxonomic level (taxa). For example: Kingdom, Phylum, Class, Order, Family, or Genus.

In [ ]:
gen = dataset$merge_taxa(taxa = "Genus")
gen

Remember that the number of OTUs will be reduced when grouped, compared to those present in the original `dataset`.

### 5.2 Clonación y modificación de datos

### 5.2 Cloning and Modifying Data

Sometimes, transformations need to be made within a data file, such as when the `gen` object was created. However, it is always important to keep the original data file to maintain order when programming.

For this, *microeco* implements the `clone` function. In the following example, the data set is first cloned, and then filtered by a column of metadata. In the second example, it is filtered by a specific bacterium in the taxonomy dataset.


#### 5.2.1 Cloning and Subsetting by Developmental State

In [ ]:
# Clone the Original Dataset

tadpoles = clone(dataset)

In [ ]:
# Select only the tadpoles from the metadata

tadpoles$sample_table <- subset(tadpoles$sample_table, life_stage == "tadpole")

In [ ]:
# Modify All Other Files to Contain Only Tadpole Information

tadpoles$tidy_dataset()

#### 5.2.2 Cloning and Subsetting by Bacteria from the Phylum Proteobacteria

In [ ]:
# Clone the Original Dataset
# You can also filter by bacteria

proteo = clone(dataset)

In [ ]:
# Filter by Proteobacteria

proteo$tax_table <- subset(proteo$tax_table, Phylum == "Proteobacteria")

In [ ]:
# Modify all other files to contain only information about Proteobacteria

proteo$tidy_dataset()

> **Important Note:** Keep in mind that when modifying the OTU table or the phylogenetic tree, diversity and abundance estimates must be recalculated. This is because the values used to estimate diversity in the dataset will no longer be the same once the OTUs or phylogeny are modified, as in the case of *Proteobacteria*. Therefore, you should use `proteo$cal_albund()` and other functions to re-estimate the diversity indices.

### 5.3 Filter by Relative Abundance

It is common for sequencing data to contain many bacteria that are underrepresented in the study unit. When performing statistical analyses, it may be necessary to select those that have a higher relative abundance compared to other bacteria. To do this, the `$filter_taxa` function is used, where the desired relative abundance threshold should 


In [ ]:
# Clone the dataset

dataset_filter <- clone(dataset)

In [ ]:
# Adjust the relative abundance threshold

dataset_filter$filter_taxa(rel_abund = 0.001)

In [ ]:
# Adapt the other files to the change

dataset_filter$tidy_dataset()

## 6. Descriptive Analysis from Relative Abundance

One of the most important aspects of data analysis is the use of graphs. Regardless of the field of study, in most cases where data is available, its presentation is usually done through bar graphs, scatter plots, and others. In this specific case, relative abundance graphs, boxplots, and heatmaps are the most common when representing bacterial composition. This section covers the creation of these graphs using *microeco*.


### 6.1 Object for Creating Graphs

A key step in creating graphs easily involves the creation of an object suitable for interpreting and adjusting the characteristics of the graphs. For this, the function `trans_abund$new` is used. This function, in addition to modifying the initial structure, requires two parameters. The first, called `taxrank`, is used to select the desired level in the taxonomy. Finally, the number of "taxa" `ntaxa` can be adjusted, which determines how many taxa will be displayed in the graphs in order of abundance.

In [ ]:
# Visualization at the Phylum Level

t1 <- trans_abund$new(dataset = dataset, taxrank = "Phylum", ntaxa = 8)

### 6.2 Relative Abundance Plot by Study Unit

After creating the object `t1`, the function `plot_bar` should be used to plot the relative abundance of each subject. Other features of the function include adjusting parameters such as `facet` in order to group individuals by the desired column from the metadata, as well as the color in which bacteria categorized as "other" should be displayed, using the `others_color` parameter. Finally, other modifications can be made, such as adjusting the legend using `xtext_keep`, among other available formatting adjustments.

In [ ]:
t1$plot_bar( facet = "life_stage",others_color = "grey",xtext_keep = F,legend_text_italic = F)

You will obtain the following result:

![barplot](./Images/barplot.png)

Just like the conventional bar graph, there are other ways to present the same information, where it is again observed that the tadpoles show a tendency towards a poorly defined composition at the phylum level, compared to juvenile and adult frogs.

In [ ]:
t1$plot_bar(bar_type = "notfull", use_alluvium = TRUE, xtext_angle = 30, xtext_size = 8, color_values = RColorBrewer::brewer.pal(8, "Set2"))

You will obtain the following result:

![barplot](./Images/alluvium.png)

### 6.3 Relative Abundance Plot by Study Group

Although presenting the information by individual is important, in certain cases it may be more practical to summarize it by the study group. To achieve this, a slight modification can be made to the original object `t1` using the `groupmean` parameter. This allows for obtaining the average per group based on one of the available columns in the metadata.

In [ ]:
t2 = trans_abund$new(dataset = dataset, taxrank = "Phylum", ntaxa = 10, groupmean = "life_stage")
t2$plot_bar(others_color = "grey70", legend_text_italic = FALSE)

You will obtain the following result:

![barplot](./Images/phyllum.png)

### 6.4 Abundance Boxplot

For this example, instead of using the Phylum level to present relative abundance, the `tax_rank` parameter is modified to visualize at the Class level. Additionally, the 4 most abundant taxa are kept in the plot using `ntaxa`.

In [ ]:
# Visualization at the Class Level

t2 = trans_abund$new(dataset = dataset, taxrank = "Class", ntaxa =4)
t2$plot_box(group = "life_stage")

You will obtain the following result:

![barplot](./Images/boxplot.png)

### 6.5 Heatmap

Heatmaps are a useful way to visualize patterns in large amounts of data. In this case, the 40 most abundant taxa at the Order level are selected, and the information is displayed according to the 3 developmental stages of each frog.

In [ ]:
# Visualization at the Order Level

t1 <- trans_abund$new(dataset = dataset, taxrank = "Order", ntaxa = 40)
t1$plot_heatmap(facet = "life_stage", xtext_keep = FALSE, withmargin = FALSE)

You will obtain the following result:

![barplot](./Images/heatmap.png)

As a conclusion from the previous graph, it can be inferred that, for example, Fusobacteriales had low abundances in adult and juvenile frogs; however, in tadpoles, this proportion was higher.

### 6.6 Pie Charts

The final relative abundance visualization in this guide is implemented using donut charts, in this case at the Class level. The `add_level` parameter of the `plot_pie` function allows adding these percentages to the chart.

In [ ]:
# Visualization at the Class Level

t1 <- trans_abund$new(dataset = dataset, taxrank = "Class", ntaxa = 5, groupmean = "life_stage")

In [ ]:
t1$plot_pie(facet_nrow = 1, add_label = TRUE)

You will obtain the following result:

![barplot](./Images/piechart.png)

## 7. Diversity Analysis

In the context of bacterial composition analysis, various techniques have been developed to quantify and analyze the variability of bacterial species present in a specific study unit. These are known as diversity indices. The purpose of this quantification is to serve as a tool that allows researchers to draw conclusions from the data and formulate hypotheses about the potential factors that could influence bacterial composition.

### 7.1 Alpha Diversity

The simplest indices to analyze are the alpha diversity indices, as they consist of only 1 metric per study unit. There are multiple indices to represent this diversity, and the function `$cal_alphadiv` allows estimating each of these. We use PD = T within the function to include the Faith's Phylogenetic Diversity index.

In [ ]:
dataset$cal_alphadiv(PD = T)

Access to the diversity indices is done through `dataset$alpha_diversity`. In the code, the `head()` function is used to display the indices for the first frogs in the dataset.

In [ ]:
head(dataset$alpha_diversity)

You will obtain the following result:

In the columns, you will observe the indices, and in the rows, the samples.

![barplot](./Images/indices.png)



#### 7.1.1 Alpha Diversity Analysis

##### 7.1.1.1 Hypothesis Testing

It is common in research contexts to study the presence of statistical differences between study groups, and various methods have been developed to compare data distributions and support or reject a proposed hypothesis. 

This document does not aim to go into detail on hypothesis testing, and it is recommended that the reader investigate its proper execution. On the other hand, to understand the alpha diversity analysis, the following example is presented: 

It is known that we can calculate the alpha diversity for each frog using metrics like the *Shannon* index. However, in order to test if there are statistically significant differences between the developmental stages, we must propose a **null hypothesis (H0)** and an **alternative hypothesis (H1)**.

Where:

- **H0:** There are no differences in the Shannon index across the developmental stages (adult, juvenile, and tadpole).

- **H1:** There are differences in the Shannon index in at least one pair of stages (adult-juvenile-tadpole).

To contrast this hypothesis test with 3 groups, the *Kruskal-Wallis* test should be used. If there are only 2 groups, a *Wilcoxon* test would be used. Depending on the case, **ANOVA** could also be considered.

In [ ]:
# Common Way to Test Hypothesis

kruskal.test(dataset$alpha_diversity$Shannon,g = dataset$sample_table$life_stage)

In [ ]:
# Quick Option with `trans_alpha` Setting Metadata Group

t1 <- trans_alpha$new(dataset = dataset, group = "life_stage")
t1$cal_diff()
t1$res_diff

# Note that for the Shannon index, the "P.unadj" is the same

> **Note:** If differences exist, they would be observed when the P.adj is less than 0.05. However, it is concluded that there is not enough statistical evidence to reject the null hypothesis, which states that there are no differences in the Shannon index between the adult, juvenile, and tadpole states.

#### 7.1.1.2 Alpha diversity index plots

In order to present the findings from the previous section, the library offers plots for the diversity index by study groups. To do this, we must have used the `cal_diff()` function on our dataset to incorporate the `plot_alpha` function. By adjusting the `measure` parameter, we can select the diversity index of interest, and the `shape` parameter adjusts the shape of the points for each group.

In [ ]:
# Shannon index plot by life stage

t1$plot_alpha(measure = "Shannon", shape = "life_stage")

You will obtain the following result:

![barplot](./Images/alpha.png)

> **Note:** To modify the "ns" appearing in the upper-left corner of the plot, please refer to the link provided at the beginning of the guide. Based on what was discussed in this section, you now have the tools needed to analyze alpha diversity using hypothesis tests and report differences using visualizations such as boxplots.

### 7.2 Beta Diversity

The process of estimating beta diversity is very similar; in this case, the data file should be named, and then `$cal_betadiv` should be used.

In [ ]:
dataset$cal_betadiv()

In [ ]:
head(data.frame(dataset$beta_diversity))[1:3,1:3]

> **Note:** For the calculation of these indices, microeco benefits from other libraries like *vegan* or *physeq*. It is important that the user consults the guide, keeping in mind that the estimation of these metrics uses various formulas and mathematical processes, which may eventually need adjustments depending on the type and amount of data available. By default, the function uses the Bray and Jaccard methods as beta diversity metrics. Although this is not explicitly mentioned in the function, it is using the following parameter: `method = c("bray", "jaccard")`. Consult the documentation of the `vegdist` function to see the available method options.

#### 7.2.1 Beta Diversity Analysis

So far, we have calculated alpha and beta diversity indices. However, the measure to represent beta diversity is not a value that belongs to a single unit, as it was in the case of alpha diversity. In this case, the OTU matrix is used to calculate distances between each pair of study units, where the goal is to represent the variability or dissimilarity between each pair of samples in order to use machine learning tools that help generate visualizations or statistical tests like PERMANOVA (see https://uw.pressbooks.pub/appliedmultivariatestatistics/chapter/permanova/) to draw conclusions from the data.

As in previous cases, an object must first be created to interpret beta diversity. For this, the function `trans_beta$new` is used. The group present in the metadata must be adjusted, as well as the `measure` parameter, which must match the method used in the `$cal_betadiv()` function (previously, Bray and Jaccard were used as methods).


In [ ]:
# Diversity interpretation object, measure must be bray or jaccard

t1 <- trans_beta$new(dataset = dataset, group = "life_stage", measure = "bray")

##### 7.2.1.1 Graphical representation of beta diversity

To graphically represent beta diversity, there are various methods such as PCoA, PCA, or non-metric multidimensional scaling (NMDS). The function `$cal_ordination` allows selecting the type of visualization based on the user's needs through the `ordination` parameter.

In [ ]:
# Set a seed so that the results match

set.seed(1)

In [ ]:
# Estimation of coordinates to represent beta diversity

t1$cal_ordination(method = "NMDS")

In [ ]:
# NMDS Plot

t1$plot_ordination(plot_color = "life_stage",plot_shape = "life_stage", plot_type = c("point", "ellipse"))

You will obtain the following result:

![barplot](./Images/beta.png)

> **Note:** When using `cal_ordination`, the function may issue a warning indicating insufficient data, suggesting that the graphical representation may not be accurate. It is recommended to visit [this link](https://uw.pressbooks.pub/appliedmultivariatestatistics/chapter/nmds/) to understand the step-by-step process of this technique.

> **Important to note:** Bray-Curtis distances are calculated from the OTU matrix, which can be accessed using `dataset$otu_table`. Keep in mind that if this matrix is modified (e.g., by creating a subset or grouping by a specific taxonomic level), the distance between samples will change, along with the visualizations of beta diversity.

In [ ]:
set.seed(1)
prueba = clone(dataset)
filo = prueba$merge_taxa(taxa = "Phylum") # Group by Phylum
filo$cal_betadiv() # Calculate beta diversity

t1$plot_ordination(plot_color = "life_stage",plot_shape = "life_stage", plot_type = c("point", "ellipse"))

You will obtain the following result:

![barplot](./Images/beta2.png)